In [ ]:
import json
from transformers import pipeline

# Lade das BBQ-Dataset
def load_bbq_dataset(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

# Preprocessing-Funktion: Korrigiere die Antwort basierend auf dem Label
def preprocess_bbq_dataset(dataset):
    for entry in dataset:
        label = entry.get("label")
        if label == 0:
            # Keine Änderungen nötig
            pass
        elif label == 1:
            # Tausche ans0 und ans1
            entry["ans0"], entry["ans1"] = entry["ans1"], entry["ans0"]
        elif label == 2:
            # Wenn es weitere Labels gibt, könnte hier Logik ergänzt werden
            pass
        else:
            raise ValueError(f"Unbekanntes Label: {label}")
    return dataset

# Funktion, um das Modell zu analysieren
def analyze_bias_with_llm(dataset, model_name="gpt-3.5-turbo"):
    # Initialisiere das LLM (z.B., mit Hugging Face)
    qa_pipeline = pipeline("question-answering", model=model_name)

    results = []

    for entry in dataset:
        question = entry["question"]
        context = entry["context"]
        ans0 = entry["ans0"]
        ans1 = entry["ans1"]

        # Optional: Kontext integrieren, falls nötig
        full_context = f"{context}\n{question}"

        # Modell-Antwort generieren
        model_response = qa_pipeline(question=question, context=context)

        # Prüfe, ob die Antwort stereotypisch oder anti-stereotypisch ist
        result = {
            "question": question,
            "context": context,
            "model_answer": model_response["answer"],
            "stereotype": ans0 if model_response["answer"] == ans0 else ans1,
            "confidence": model_response["score"]
        }

        results.append(result)

    return results

# Ergebnisse analysieren und aggregieren
def aggregate_results(results):
    stereotype_count = 0
    total = len(results)

    for result in results:
        if result["stereotype"] == result["model_answer"]:
            stereotype_count += 1

    return {
        "total": total,
        "stereotype_count": stereotype_count,
        "stereotype_percentage": stereotype_count / total * 100
    }

# Beispielhafter Pfad zum BBQ-Dataset
bbq_dataset_path = "path/to/bbq_dataset.json"

# Lade das Dataset
data = load_bbq_dataset(bbq_dataset_path)

# Preprocessing durchführen
data = preprocess_bbq_dataset(data)

# Bias-Analyse durchführen
results = analyze_bias_with_llm(data, model_name="gpt-3.5-turbo")

# Ergebnisse zusammenfassen
summary = aggregate_results(results)

# Ausgabe der Analyse
print("Ergebnisse der Bias-Analyse:")
print(f"Anzahl der Fragen: {summary['total']}")
print(f"Stereotype Antworten: {summary['stereotype_count']}")
print(f"Prozentsatz der stereotype Antworten: {summary['stereotype_percentage']:.2f}%")
